In [1]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import EthereumQuerier, EthereumProcessor, EthereumPipeline
import logging
from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations, SQLDatabase

logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')


In [2]:
mongodb = MongoDatabase()
sql = SQLDatabase()

2025-01-17 00:21:28,498 - INFO - Connected to MongoDB database: blockchain
2025-01-17 00:21:28,507 - INFO - Indexes created for collection: Solana
2025-01-17 00:21:28,508 - INFO - Indexes created for collection: Bitcoin
2025-01-17 00:21:28,510 - INFO - Indexes created for collection: Ethereum
2025-01-17 00:21:28,511 - INFO - Indexes created for collection: BNB
2025-01-17 00:21:28,512 - INFO - Indexes created for collection: XRP
2025-01-17 00:21:28,513 - INFO - Indexes created for collection: Base
2025-01-17 00:21:28,551 - INFO - Applying schema from C:\Users\Nicko\vscodeprojects\blockchain_tracker\python\database\sql\schema.sql
2025-01-17 00:21:28,556 - INFO - Schema applied successfully.


In [3]:
querier = EthereumQuerier()
processor = EthereumProcessor(sql, mongodb, querier)

block = await querier.get_block(21585277)



2025-01-17 00:21:28,563 - INFO - Initialized Ethereum querier
2025-01-17 00:21:28,565 - INFO - Initializing WebSocketHandler for Ethereum
2025-01-17 00:21:28,566 - INFO - Initializing EVMWebSocketHandler for Ethereum
2025-01-17 00:21:28,568 - INFO - Initialized Ethereum processor
2025-01-17 00:21:28,866 - INFO - Fetching block 21585277


In [5]:
block

AttributeDict({'baseFeePerGas': 3412600388,
 'blobGasUsed': 0,
 'difficulty': 0,
 'excessBlobGas': 65404928,
 'extraData': HexBytes('0x'),
 'gasLimit': 30000000,
 'gasUsed': 5369950,
 'hash': HexBytes('0xb968c199f308b54c4c92b5defbb1233a57710685fcaf2cc3789a2f4800c59cd3'),
 'logsBloom': HexBytes('0x8063402305803888801080c2801139143881a4882620600082090c330da2224406894004cac084810a04984054144d15625080a5297220050101390286ac2a210058080a440042494a202118924070a90300084100473440f8df08208e2150210a1991548388052b264802d038004d0a02090100c348260393a000900058280b0122088810024049007940d31b2222261918251105410048088c24d4609342020b4489000481a20608ac42c48921010002c11000a81005282430008120280083211030c2242084c062000c5202898a05408c80036d2408d420810142600260144436e05daaa2801182c130830022d8380a303641428286ec80001c01000c1445'),
 'miner': '0x388C818CA8B9251b393131C08a736A67ccB19297',
 'mixHash': HexBytes('0xf18b8f0b56dfa9fe6c60bfa2c318b2c2caebade7f26d6be919f70d3d56e4e2e9'),
 'nonce': HexBytes('0x0000000000000000

In [4]:
mongodb = MongoDatabase()
mongodb_query_ops = MongoQueryOperations(mongodb)

query = mongodb_query_ops.get_block_by_number(21585277, 'Ethereum')

2025-01-17 00:21:29,496 - INFO - Connected to MongoDB database: blockchain
2025-01-17 00:21:29,522 - INFO - Indexes created for collection: Solana
2025-01-17 00:21:29,524 - INFO - Indexes created for collection: Bitcoin
2025-01-17 00:21:29,526 - INFO - Indexes created for collection: Ethereum
2025-01-17 00:21:29,527 - INFO - Indexes created for collection: BNB
2025-01-17 00:21:29,529 - INFO - Indexes created for collection: XRP
2025-01-17 00:21:29,530 - INFO - Indexes created for collection: Base
2025-01-17 00:21:29,530 - ERROR - Error retrieving block Ethereum from 21585277 collection in MongoDB: Invalid network name provided for MongoDB collection.


TypeError: 'NoneType' object is not subscriptable

In [17]:
def decode_hex(value):
    """
    Decode a hexadecimal string to an integer if it's an Ethereum-style integer (e.g., block numbers, gas values).
    Does not decode long hashes or other non-integer hex values.
    :param value: Hexadecimal string (e.g., '0x677df92f') or other types.
    :return: Decoded integer or original value if not a valid short hex integer.
    """
    if isinstance(value, str) and value.startswith("0x"):
        if value == '0x':
            return None
        
        # Only decode if the hex string is short (e.g., block numbers, gas, timestamps)
        return int(value, 16)
    return value  # Return original value if not a short hex integer


In [18]:
from web3 import Web3
# Parallelization of the tasks
def reconcile_logs_with_transactions(block):
    
    transactions = block["transactions"]
    w3 = Web3(Web3.HTTPProvider(Settings.ETHEREUM_ENDPOINT))
    
    # Fetch logs for the block
    logs = w3.eth.get_logs({
        "fromBlock": decode_hex(block['number']),
        "toBlock": decode_hex(block['number'])
    })

    # Group logs by transaction hash
    logs_by_transaction = {}
    for log in logs:
        tx_hash = log["transactionHash"].to_0x_hex()
        if tx_hash not in logs_by_transaction:
            logs_by_transaction[tx_hash] = []
        logs_by_transaction[tx_hash].append(dict(log))

    # Analyze the results
    tx_with_logs = len(logs_by_transaction)
    tx_without_logs = len(transactions) - tx_with_logs

    
    
    print(f"Total transactions in block: {len(transactions)}")
    print(f"Transactions with logs: {tx_with_logs}")
    print(f"Transactions without logs: {tx_without_logs}")

    # Return grouped logs for further analysis
    return logs_by_transaction



In [19]:
logs_by_transaction = reconcile_logs_with_transactions(data)

Total transactions in block: 91
Transactions with logs: 49
Transactions without logs: 42


In [20]:
logs_by_transaction['0x2d7f82551475f9c88a735633c8ad5a8bc7877f51f28355ffc583a5598b8c144e']


[{'address': '0xbf5495Efe5DB9ce00f80364C8B423567e58d2110',
  'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'),
   HexBytes('0x000000000000000000000000469d02d03bd26c12fc02407910a63b856151e867'),
   HexBytes('0x000000000000000000000000c59336d8edda9722b4f1ec104007191ec16f7087')],
  'data': HexBytes('0x000000000000000000000000000000000000000000000000000a07eb724451d7'),
  'blockNumber': 21585277,
  'transactionHash': HexBytes('0x2d7f82551475f9c88a735633c8ad5a8bc7877f51f28355ffc583a5598b8c144e'),
  'transactionIndex': 45,
  'blockHash': HexBytes('0xb968c199f308b54c4c92b5defbb1233a57710685fcaf2cc3789a2f4800c59cd3'),
  'logIndex': 49,
  'removed': False},
 {'address': '0xbf5495Efe5DB9ce00f80364C8B423567e58d2110',
  'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
   HexBytes('0x000000000000000000000000469d02d03bd26c12fc02407910a63b856151e867'),
   HexBytes('0x000000000000000000000000c59336d8edda9722b4f1ec1040071

In [21]:
import requests
import json
from web3 import Web3
from eth_abi.codec import ABICodec
from eth_abi.registry import registry

abi_codec = ABICodec(registry)

def fetch_abi_from_explorer(contract_address):

    params = {
        "module": "contract",
        "action": "getabi",
        "address": contract_address,
        "apikey": Settings.ETHERSCAN_API_KEY
    }

    response = requests.get("https://api.etherscan.io/api", params=params)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "1":
            return json.loads(data["result"])
    return None



def get_event_signature_hash(event_abi):
    signature = event_abi["name"] + "(" + ",".join(i["type"] for i in event_abi["inputs"]) + ")"
    return Web3.keccak(text=signature).hex()

def extract_events_from_abi(abi):
    # Filter ABI for event definitions
    return [item for item in abi if item["type"] == "event"]

def map_signatures_to_events(abi):
    # Extract event definitions
    events = [item for item in abi if item["type"] == "event"]

    # Create a mapping from hashed signature to event ABI
    signature_to_event = {}
    for event in events:
        signature = event["name"] + "(" + ",".join([input["type"] for input in event["inputs"]]) + ")"
        hashed_signature = Web3.keccak(text=signature).hex()
        signature_to_event[hashed_signature] = event

    return signature_to_event


def decode_log(log, event_abi):
    # Decode indexed parameters from topics
    indexed_inputs = [i for i in event_abi["inputs"] if i["indexed"]]
    decoded_topics = [
        Web3.codec.decode_single(i["type"], bytes.fromhex(topic[2:]))
        for topic, i in zip(log["topics"][1:], indexed_inputs)
    ]

    # Decode non-indexed parameters from data
    non_indexed_inputs = [i for i in event_abi["inputs"] if not i["indexed"]]
    decoded_data = abi_codec.decode(
        [i["type"] for i in non_indexed_inputs],
        bytes.fromhex(log["data"][2:])
    )

    # Combine decoded parameters
    decoded_event = {
        i["name"]: value for i, value in zip(indexed_inputs + non_indexed_inputs, decoded_topics + list(decoded_data))
    }
    decoded_event["event"] = event_abi["name"]
    return decoded_event

def match_logs_to_events(logs, signature_to_event):
    matched_logs = []
    for log in logs:
        # First topic is the event signature hash
        event_signature = log["topics"][0].hex()
        if event_signature in signature_to_event:
            event_abi = signature_to_event[event_signature]
            matched_logs.append({"log": log, "event_abi": event_abi})
        else:
            matched_logs.append({"log": log, "event_abi": None})  # Unknown event
    return matched_logs




In [23]:
from blockchain_tracker.notebooks.transactions_tests.ethereum.decoder import LogDecoder

log_decoder = LogDecoder(Settings.ETHERSCAN_API_KEY)



In [24]:
log_decoder.process_transaction_logs(logs_by_transaction)

Contract: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, Event: ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, Count: 2
Contract: 0xdCD48419bD5Cd9d1b097695F2af4Ee125aADF84F, Event: 2a08a2bd2798f0aae9a843f0f4ad4de488c1b3d5f04049940cfed736ad69fb97, Count: 1
Contract: 0xc3CA38091061e3E5358A52d74730F16C60cA9c26, Event: 36c96ccc320eab9b6da1de6cc1e8ebc52a51cb114710b9c7db204cdad3067ce1, Count: 2
Contract: 0x9a15bB3a8FEc8d0d810691BAFE36f6e5d42360F7, Event: 4d8aead3491b7eba4b5c7a65fc17e493b9e63f9e433522fc5f6a85a168fc9d36, Count: 1
Contract: 0x5c7BCd6E7De5423a257D81B442095A1a6ced35C5, Event: a123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f, Count: 1
Contract: 0xce16F69375520ab01377ce7B88f5BA8C48F8D666, Event: 5844b8bbe3fd2b0354e73f27bfde28d2e6d991f14139c382876ec4360391a47b, Count: 1
Contract: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, Event: 8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925, Count: 1
Contract: 0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f

[{'transactionHash': '0x7ffd4cee4f19e6ed535817943c6a9cb2040a5c8c2f37ded19d88d9c0edf6d5fb',
  'actions': [{'contract': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
    'details': {'address': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
     'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
      HexBytes('0x0000000000000000000000007eb6c83ab7d8d9b8618c0ed973cbef71d1921ef2'),
      HexBytes('0x000000000000000000000000c03ca55da3179a572a8d9a78d06101babfe801b8')],
     'data': HexBytes('0x00000000000000000000000000000000000000000000000000000004a7fdfe70'),
     'blockNumber': 21585277,
     'transactionHash': HexBytes('0x7ffd4cee4f19e6ed535817943c6a9cb2040a5c8c2f37ded19d88d9c0edf6d5fb'),
     'transactionIndex': 8,
     'blockHash': HexBytes('0xb968c199f308b54c4c92b5defbb1233a57710685fcaf2cc3789a2f4800c59cd3'),
     'logIndex': 0,
     'removed': False},
    'type': 'Heuristic Decoded Event'}]},
 {'transactionHash': '0x3c75c095fdaa82b6b79bdd78c2cce6d

In [25]:
log_decoder

In [ ]:
output = process_logs_by_event(logs_by_transaction)



In [26]:
ABI = fetch_abi_from_explorer(logs_by_transaction['0x2d7f82551475f9c88a735633c8ad5a8bc7877f51f28355ffc583a5598b8c144e'][2]['address'])


In [27]:
signature_to_event = map_signatures_to_events(ABI)


In [28]:
signature_to_event


{'7e644d79422f17c01e4894b5f4f588d331ebfa28653d42ae832dc59e38c9798f': {'anonymous': False,
  'inputs': [{'indexed': False,
    'internalType': 'address',
    'name': 'previousAdmin',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'newAdmin',
    'type': 'address'}],
  'name': 'AdminChanged',
  'type': 'event'},
 '1cf3b03a6cf19fa2baba4df148e9dcabedea7f8a5c07840e207e5c089be95d3e': {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'beacon',
    'type': 'address'}],
  'name': 'BeaconUpgraded',
  'type': 'event'},
 'bc7cd75a20ee27fd9adebab32041f755214dbc6bffa90cc0225b39da2e5c2d3b': {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'implementation',
    'type': 'address'}],
  'name': 'Upgraded',
  'type': 'event'}}

In [29]:
transaction = [tx for tx in data['transactions'] if tx['hash'] == '0x2d7f82551475f9c88a735633c8ad5a8bc7877f51f28355ffc583a5598b8c144e']
transaction[0]

{'blockHash': '0xb968c199f308b54c4c92b5defbb1233a57710685fcaf2cc3789a2f4800c59cd3',
 'blockNumber': '0x1495d7d',
 'from': '0x469d02d03bd26c12fc02407910a63b856151e867',
 'gas': '0x9120d',
 'gasPrice': '0x10702f044',
 'maxFeePerGas': '0x15bef0b05',
 'maxPriorityFeePerGas': '0x3b9aca00',
 'hash': '0x2d7f82551475f9c88a735633c8ad5a8bc7877f51f28355ffc583a5598b8c144e',
 'input': '0x81b4e8b4000000000000000000000000000000000000000000000000000000000000e708000000000000000000000000469d02d03bd26c12fc02407910a63b856151e86700000000000000000000000000000000000000000000000001bcb766f0fbae297375706572627269646765',
 'nonce': '0x3c',
 'to': '0xc59336d8edda9722b4f1ec104007191ec16f7087',
 'transactionIndex': '0x2d',
 'value': '0x2c33020e29670',
 'type': '0x2',
 'accessList': [],
 'chainId': '0x1',
 'v': '0x0',
 'r': '0x4de13444dab57a5a4c8489269637172ac576579042a294850152fa0cb2378fa5',
 's': '0x6ed1a04fa4be15d6ad390151e1a130b7359b6f6e76e724b225afb3f6493fcea1',
 'yParity': '0x0'}

In [30]:
decode_hex(data['transactions'][0]['input'])

In [31]:
counter = 0
for tx in data['transactions']:
    if tx['input'] == '0x':
        counter += 1
counter


41